# A- Write and implement an algorithm to automatically collect as much as metadata possible of the 250 titles and store them in a CSV file.

## 1- Import libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')
import bs4
import pandas as pd
import numpy as np
import requests
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

## 2- Storing movie name and link

In [10]:
movie_name=[] # for storing movie names
movie_link=[] # for storing their IMDB links
Data = BeautifulSoup(urlopen('http://www.imdb.com/chart/top?ref_=nv_mv_250_6'))
df = Data.find('tbody', {'class':'lister-list'}).find_all('td', {'class':'titleColumn'})
for i in range(0,250):
    movie_link.append('https://www.imdb.com'+Data.find('tbody', {'class':'lister-list'}).find_all('td', {'class':'titleColumn'})[i].a.attrs['href'])
for links in df:
    movie_name.append(str(links.a.text))

## 3- Collect metadata for 250 movies

In [11]:
imdb=[]      # IMDB rating
run_time=[]  # Running time of the movie
dirct=[]     # Diector
rel_date=[]  # Release Date
country=[]   # Country
no_user=[]   # No. of users who had given ratings
meta=[]       # Metascore
summary=[]   # Summary
first_writer=[]  #first writer
first_actor= [] #first_actor
genre=[]    #genre
Censor_Board_Rating=[]  #Censor_Board_Rating


def links_data(links):
    Data = BeautifulSoup(urlopen(links))
    imdb.append(Data.find('span', {'itemprop':'ratingValue'}).text) #imdb rating
    run_time.append(Data.find('div', {'class':'subtext'}).time.text.strip()) # running time
    dirct.append(Data.find_all('div', {'class':'credit_summary_item'})[0].a.text) #director
    rel_date.append(Data.find('a', {'title':'See more release dates'}).text.strip()) #release date
    country.append(Data.find('div', {'class':'article', 'id':'titleDetails'}).find_all('div')[1].find('a').text)
    no_user.append(Data.find('span', {'class':'small', 'itemprop':'ratingCount'}).text) #no. of user rating
    try:
        meta.append(Data.find('div', {'class':'metacriticScore score_favorable titleReviewBarSubItem'}).text.strip()) #metascore
    except AttributeError:
        meta.append(np.nan)
    summary.append(Data.find("div",{'class':'summary_text'}).text.strip())
    first_writer.append(Data.find_all('div', {'class':'credit_summary_item'})[1].a.text) #first_writer
    first_actor.append(Data.find_all('div', {'class':'credit_summary_item'})[2].a.text) #first_actor   
    genre.append(Data.find_all('div', {'class':'subtext'})[0].a.text) #genre
    Censor_Board_Rating.append(Data.find_all('div', {'class':'subtext'})[0].text) #Censor_Board_Rating 
  
for i in range(0,250):
    links_data(movie_links[i])

## 4- Create dataframe and clean some data

In [12]:
movie_list = pd.DataFrame({'Movie Name':movie_name[0:250], 'Movie Link':movie_links[0:250], 'IMDB Rating':imdb, 'Running Time':run_time, 'Director':dirct, 'Release Date':rel_date,  'Country':country, 'Votes':no_user,'Meta score':meta, 'Summary':summary,'First Writer':first_writer,'First Actor':first_actor,'Genre':genre,'Censor_Board_Rating':Censor_Board_Rating})
movie_list['Censor_Board_Rating']= movie_list['Censor_Board_Rating'].astype("str")
new= movie_list['Censor_Board_Rating'].str.split("|", n = 1, expand = True) 
movie_list['Censor_Board_Rating']= new[0]
movie_list['Censor_Board_Rating'] = movie_list['Censor_Board_Rating'].replace('\n','', regex=True)
movie_list['Censor_Board_Rating'] = movie_list['Censor_Board_Rating'].str.lstrip()

## 5- Display the data frame

In [13]:
movie_list.head(10)

,Movie Name,Movie Link,IMDB Rating,Running Time,Director,Release Date,Country,Votes,Meta score,Summary,First Writer,First Actor,Genre,Censor_Board_Rating
0,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,9.3,2h 22min,Frank Darabont,14 October 1994 (USA),USA,"2,351,862",80,Two imprisoned men bond over a number of years...,Stephen King,Tim Robbins,Drama,14A
1,The Godfather,https://www.imdb.com/title/tt0068646/,9.2,2h 55min,Francis Ford Coppola,24 March 1972 (Canada),USA,"1,627,409",100,An organized crime dynasty's aging patriarch t...,Mario Puzo,Marlon Brando,Crime,18A
2,The Godfather: Part II,https://www.imdb.com/title/tt0071562/,9.0,3h 22min,Francis Ford Coppola,18 December 1974 (Canada),USA,"1,134,368",90,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Crime,PG
3,The Dark Knight,https://www.imdb.com/title/tt0468569/,9.0,2h 32min,Christopher Nolan,18 July 2008 (Canada),USA,"2,311,546",84,When the menace known as the Joker wreaks havo...,Jonathan Nolan,Christian Bale,Action,PG
4,12 Angry Men,https://www.imdb.com/title/tt0050083/,9.0,1h 36min,Sidney Lumet,13 April 1957 (Canada),English,"692,966",96,A jury holdout attempts to prevent a miscarria...,Reginald Rose,Henry Fonda,Crime,PG
5,Schindler's List,https://www.imdb.com/title/tt0108052/,8.9,3h 15min,Steven Spielberg,25 December 1993 (Canada),USA,"1,217,513",94,"In German-occupied Poland during World War II,...",Thomas Keneally,Liam Neeson,Biography,PA
6,The Lord of the Rings: The Return of the King,https://www.imdb.com/title/tt0167260/,8.9,3h 21min,Peter Jackson,17 December 2003 (Canada),New Zealand,"1,648,173",94,Gandalf and Aragorn lead the World of Men agai...,J.R.R. Tolkien,Elijah Wood,Action,14A
7,Pulp Fiction,https://www.imdb.com/title/tt0110912/,8.9,2h 34min,Quentin Tarantino,14 October 1994 (Canada),USA,"1,833,113",94,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,John Travolta,Crime,16+
8,"Il buono, il brutto, il cattivo",https://www.imdb.com/title/tt0060196/,8.8,2h 41min,Sergio Leone,23 December 1966 (Italy),Italian,"690,743",90,A bounty hunting scam joins two men in an unea...,Luciano Vincenzoni,Clint Eastwood,Western,14+
9,The Lord of the Rings: The Fellowship of the Ring,https://www.imdb.com/title/tt0120737/,8.8,2h 58min,Peter Jackson,19 December 2001 (Canada),New Zealand,"1,667,323",92,A meek Hobbit from the Shire and eight compani...,J.R.R. Tolkien,Elijah Wood,Action,14A


## Save the data frame as .csv file

In [81]:
movie_list.to_csv('abeer_badawi.csv')